In [41]:
import pandas as pd
import numpy as np

In [151]:
from openskill.models import PlackettLuce, BradleyTerryFull

targetElo = 1000
baseElo = 500
targetSeason = 's24'
model = PlackettLuce() # beta=25.0/120.0

In [70]:
teamRegions = {'Hawaii': 'PCCSC', 'Brown': 'NEISA', 'Southern Cal': 'PCCSC', 'Salve Regina': 'NEISA', 'UC Santa Barbara': 'PCCSC', 'Cal Poly': 'PCCSC', 'Washington': 'NWICSA', 'Channel Islands': 'PCCSC', 'UC San Diego': 'PCCSC', 'British Columbia': 'NWICSA', 'UC Los Angeles': 'PCCSC', 'Westmont College': 'PCCSC', 'Arizona State': 'PCCSC', 'Texas A&M Galveston': 'SEISA', 'Texas A&M': 'SEISA', 'Tulane': 'SEISA', 'Rice': 'SEISA', 'Texas': 'SEISA', 'Oklahoma State': 'SEISA', 'Texas A&M C. Christ': 'SEISA', 'Central Oklahoma': 'SEISA', 'Notre Dame': 'MCSA', 'Jacksonville': 'SAISA', 'Florida': 'SAISA', 'Tennessee': 'SAISA', 'Rollins': 'SAISA', 'North Carolina State': 'SAISA', 'Georgia Tech': 'SAISA', 'Auburn': 'SAISA', 'Charleston': 'SAISA', 'South Florida': 'SAISA', 'Old Dominion': 'MAISA', 'Eckerd': 'SAISA', 'Florida State': 'SAISA', 'U. Miami': 'SAISA', 'UW Milwaukee': 'MCSA', 'Stony Brook': 'MAISA', 'Duke': 'SAISA', 'Clemson': 'SAISA', 'U South Carolina': 'SAISA', 'UNC Wilmington': 'SAISA', 'Georgia': 'SAISA', 'Berkeley': 'PCCSC', 'CSU Long Beach': 'PCCSC', 'Monterey Bay': 'PCCSC', 'UC Irvine': 'PCCSC', 'UC Davis': 'PCCSC', 'Rhode Island': 'NEISA', 'Georgetown': 'MAISA', 'Dartmouth': 'NEISA', 'MIT': 'NEISA', 'George Washington': 'MAISA', 'Navy': 'MAISA', 'Fordham': 'MAISA', 'Northeastern': 'NEISA', 'Christopher Newport': 'MAISA', 'Victoria': 'NWICSA', 'Boston University': 'NEISA', 'Miami University': 'MCSA', 'Hampton': 'MAISA', 'Virginia': 'MAISA', 'Stevens': 'MAISA', 'Columbia': 'MAISA', 'NY Maritime': 'MAISA', 'Kings Point': 'MAISA', "St. Mary's": 'MAISA', 'Maryland': 'MAISA', 'Virginia Tech': 'MAISA', 'Drexel': 'MAISA', 'Maryland/Baltimore': 'MAISA', 'Buffalo': 'MAISA', 'UC Santa Cruz': 'PCCSC', 'Santa Clara': 'PCCSC', 'Wisconsin': 'MCSA', 'Michigan': 'MCSA', 'Washington College': 'MAISA', 'Minnesota': 'MCSA', 'Yale': 'NEISA', 'Hobart & William': 'MAISA', 'Vermont': 'NEISA', 'Connecticut College': 'NEISA', 'Harvard': 'NEISA', 'Roger Williams': 'NEISA', 'Syracuse': 'MAISA', 'Tufts': 'NEISA', 'Middlebury': 'NEISA', 'New College': 'SAISA', 'William and Mary': 'MAISA', 'Gannon': 'MAISA', 'Boston College': 'NEISA', 'Stanford': 'PCCSC', 'Bowdoin': 'NEISA', 'Lewis & Clark': 'NWICSA', 'Monmouth': 'MAISA', 'American': 'MAISA', 'Michigan State': 'MCSA', 'Hope': 'MCSA', 'Western Michigan': 'MCSA', 'Toledo': 'MCSA', 'Ohio State': 'MCSA', 'Mass Maritime': 'NEISA', 'Coast Guard': 'NEISA', 'Bates': 'NEISA', 'Fairfield': 'NEISA', 'Sacred Heart': 'NEISA', 'Wentworth Institute': 'NEISA', 'Providence': 'NEISA', 'Iowa State': 'MCSA', 'Iowa': 'MCSA', 'Indiana': 'MCSA', 'Davidson': 'SAISA', 'Oregon State': 'NWICSA', 'Western Washington': 'NWICSA', 'U. Rochester': 'MAISA', 'Army': 'MAISA', 'New Hampshire': 'NEISA', 'U. Connecticut': 'NEISA', 'UMass Dartmouth': 'NEISA', 'Wesleyan': 'NEISA', 'U. Mass/ Amherst': 'NEISA', 'U New England': 'NEISA', 'Denison': 'MCSA', 'Northern Michigan': 'MCSA', 'Ohio': 'MCSA', 'Pennsylvania': 'MAISA', 'Villanova': 'MAISA', 'Maine Maritime': 'NEISA', 'Michigan Tech': 'MCSA', 'Illinois': 'MCSA', 'Chicago': 'MCSA', 'Northwestern': 'MCSA', 'Grand Valley State': 'MCSA', 'Washington U': 'MCSA', 'Marquette': 'MCSA', 'Lake Forest': 'MCSA', 'Cornell': 'MAISA', 'Oregon': 'NWICSA', 'Portland State': 'NWICSA', 'Princeton': 'MAISA', "Queen's": 'MAISA', 'Penn State': 'MAISA', 'Ocean County': 'MAISA', 'Delaware': 'MAISA', 'Rutgers': 'MAISA', 'Worcester Polytech': 'NEISA', 'Emmanuel College': 'NEISA', "St. John's": 'MAISA', 'U Pittsburgh': 'MAISA', 'Webb Institute': 'MAISA', 'McGill': 'NEISA', 'Citadel': 'SAISA', 'Colgate': 'MAISA', 'Catholic U America': 'MAISA', 'Loyola College': 'MAISA', 'Ottawa': 'MAISA', 'Royal Military': 'MAISA', 'Dalhousie': 'NEISA', 'U Toronto': 'MAISA', 'New Orleans': 'SEISA', 'Kansas': 'SEISA', 'Bentley': 'NEISA', 'Brandeis': 'NEISA', 'Cal Maritime': 'PCCSC', 'San Diego State': 'PCCSC', 'Loyola': 'SEISA', 'North Texas': 'SEISA', 'Vanderbilt': 'SAISA', 'Purdue': 'MCSA', 'North Carolina': 'SAISA', 'Hillsdale': 'MCSA', 'Amherst': 'NEISA', 'Williams': 'NEISA', 'Hamilton': 'MAISA', 'Rochester': 'MAISA', 'Wellesley': 'NEISA', 'Hosei Univerisity': 'GUEST', 'Colorado': 'SEISA', 'John Carroll': 'MCSA', 'U.  Mass/ Boston': 'NEISA', 'Mercyhurst': 'MAISA', 'Penn State Behrend': 'MAISA', 'Indiana U Pennsylvan': 'MAISA', 'U Nebraska': 'MCSA', 'U Maine': 'NEISA', 'Texas Christian': 'SEISA', 'Embry-Riddle': 'SAISA', 'Palm Beach Atlantic': 'SAISA', 'U of Central Florida': 'SAISA', 'Baldwin-Wallace': 'MCSA', "Saint Mary's College": 'MCSA', 'Olin': 'NEISA', 'Baylor': 'SEISA', 'Texas Tech': 'SEISA', 'Wake Forest': 'SAISA', 'Georgia Southern': 'SAISA', 'East Carolina': 'SAISA', 'Florida Tech': 'SAISA', 'Saint Thomas': 'MCSA', 'Cincinnati': 'MCSA', 'Florida Gulf Coast': 'SAISA', 'Saginaw Valley': 'MCSA', 'Coastal Georgia': 'SAISA', 'Cleveland State': 'MCSA', 'Sewanee': 'SAISA', 'Case Western': 'MCSA', 'Oklahoma': 'SEISA', 'Gonzaga': 'PCCSC'}


In [177]:
df_races = pd.read_json("allseasonsteamraces.json")
df_sailor_info = pd.read_json("sailor_data2.json")
df_sailorTRinfo = pd.read_json('trSailorInfoAll.json')
df_races = df_races.sort_values(['date', 'raceNum']).reset_index(drop=True)
df_sailorTRinfo

,name,year,link,key
0,Hugh MacGillivray,18,hugh-macgillivray,hugh-macgillivray
1,Eliza Garry,17,eliza-garry,eliza-garry
2,Carter Pemberton,20,carter-pemberton,carter-pemberton
3,Paige Dunlevy,20,paige-dunlevy,paige-dunlevy
4,Ragna Agerup,20,ragna-agerup,ragna-agerup
...,...,...,...,...
4164,Anthony Torrez,25,anthony-torrez,anthony-torrez
4165,Jack Blackman,24,jack-blackman,jack-blackman
4166,Genaro Flores - Machorro,25,genaro-flores-machorro,genaro-flores-machorro
4167,Maggie McEachen,27,maggie-mceachen,maggie-mceachen


In [158]:
class Sailor:
    def __init__(self, name, key, year, links, teams=[], seasons=[], skipperRank=0, crewRank=0,skipperRating=baseElo, crewRating=baseElo, races=[], gender=""):
        self.name = name
        self.key = key
        self.gender = gender
        self.year = year
        self.links = links
        self.teams = teams
        self.races = []
        self.seasons={'Skipper': [], 'Crew': []}
        
        self.skipperRankTR = 0
        self.crewRankTR = 0
        self.womenSkipperRankTR = 0
        self.womenCrewRankTR = 0
        
        self.wtsr = model.rating(name=name)
        self.wtcr = model.rating(name=name)
        self.tsr = model.rating(name=name)
        self.tcr = model.rating(name=name)

In [79]:
def createPeople():
    people = {}
    
    for _,person in df_sailorTRinfo.iterrows():
        gender = ''
        key = person['key']
        if key in list(df_sailor_info['link']):
            data = df_sailor_info.loc[df_sailor_info['link'] == key]
            if len(data) > 0:
                gender = data['gender'].iat[0]
        
        people[key] = Sailor(person['name'], 
                                       key, 
                                       person['year'],
                                       [person['link']], gender=gender)
    
    return people

In [178]:
people = createPeople()
i = 0
for _,raceRow in df_races.iterrows():
    i += 1
    # print(raceRow)
    for type in ['Skipper', 'Crew']:
        
        womens = False
        
        teamAName = raceRow['teamAName']
        teamAKeys = [boat[type.lower() + 'Key'] if boat[type.lower() + 'Key'] is not None else 'Unknown' for boat in raceRow['teamABoats']] # the sailor keys
        teamANames = [boat[type.lower() + 'Name'] for boat in raceRow['teamABoats']] # the sailor keys
        teamARacers = [people[key] for key in teamAKeys if 'Unknown' not in key]
        
        teamBName = raceRow['teamBName']
        teamBKeys = [boat[type.lower() + 'Key'] if boat[type.lower() + 'Key'] is not None else 'Unknown' for boat in raceRow['teamBBoats']] # the sailor keys
        teamBNames = [boat[type.lower() + 'Name'] for boat in raceRow['teamBBoats']] # the sailor keys
        teamBRacers = [people[key] for key in teamBKeys if 'Unknown' not in key]
        
        genders = [p.gender for p in teamARacers + teamBRacers]
        womenCount = sum([1 if g == "F" else 0 for g in genders])
        womens = 'M' not in genders and womenCount >= 4
        
        teamARatings = []
        if womens:
            teamARatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamARacers]
        else:
            teamARatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamARacers]
            
        teamBRatings = []
        if womens:
            teamBRatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamBRacers]
        else:
            teamBRatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamBRacers]
        
        startingARating = np.round([r.ordinal(target=targetElo, alpha=200 / model.sigma) for r in teamARatings],1)
        startingBRating = np.round([r.ordinal(target=targetElo, alpha=200 / model.sigma) for r in teamBRatings],1)
        
        if len(teamARatings) < 1 or len(teamBRatings) < 1:
            print("not enough sailors in this race, skipping", raceRow['raceID'])
            continue
        
        ratings = model.rate([teamARatings, teamBRatings], 
                             ranks=[1 if raceRow['teamAOutcome'] == 'win' else 2, 1 if raceRow['teamBOutcome'] == 'win' else 2])
        
        for team, name, newRatings in zip([teamARacers, teamBRacers], [teamAName, teamBName], ratings):
            for racer, new_rating in zip(team, newRatings):
                # print(new_rating.ordinal(target=targetElo, alpha=200 / model.sigma))
                racer.teams = [name]
                if raceRow['raceID'].split("/")[0] not in racer.seasons[type]:
                    racer.seasons[type].append(raceRow['raceID'].split("/")[0])
                if type == 'Skipper':
                    if womens:
                        racer.wtsr = new_rating
                    else: 
                        racer.tsr = new_rating
                else:
                    if womens:
                        racer.wtcr = new_rating
                    else:
                        racer.tcr = new_rating
                        
        # if womens:
        #     teamARatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamARacers]
        # else:
        #     teamARatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamARacers]
            
        # if womens:
        #     teamBRatings = [r.wtsr if type == 'Skipper' else r.wtcr for r in teamBRacers]
        # else:
        #     teamBRatings = [r.tsr if type == 'Skipper' else r.tcr for r in teamBRacers]
            
        # if type == 'Skipper':
        #     # print(womens)
        #     # print(ratings)
        #     print(raceRow['teamAOutcome'], raceRow['teamBOutcome'], [1 if raceRow['teamAOutcome'] == 'win' else 2, 1 if raceRow['teamBOutcome'] == 'win' else 2], type) 
        #     endingARatings = np.round([r.ordinal(target=targetElo, alpha=200 / model.sigma) for r in teamARatings], 1)
        #     endingBRatings = np.round([r.ordinal(target=targetElo, alpha=200 / model.sigma) for r in teamBRatings], 1)
        #     print(teamAName,startingARating,"->",endingARatings)
        #     print(teamBName,startingBRating,"->",endingBRatings)
        
        for racer in teamARacers:
            newRating = 0
            if type == 'Skipper':
                newRating = racer.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma) if womens else racer.tsr.ordinal(target=targetElo, alpha=200 / model.sigma)
            else:
                newRating = racer.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma) if womens else racer.tcr.ordinal(target=targetElo, alpha=200 / model.sigma)

            racer.races.append({'raceID': raceRow['raceID'], 'raceNum': raceRow['raceNum'], 'round':  raceRow['round'], 
                                'pos': type,
                                'type': 'womens' if womens else 'open',
                                'opponentTeam': raceRow['teamBName'], 
                                'opponentNick': raceRow['teamBNick'],
                                'score': raceRow['teamAScore'],
                                'outcome': raceRow['teamAOutcome'], 
                                'newRating': newRating})
        for racer in teamBRacers:
            newRating = 0
            if type == 'Skipper':
                newRating = racer.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma) if womens else racer.tsr.ordinal(target=targetElo, alpha=200 / model.sigma)
            else:
                newRating = racer.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma) if womens else racer.tcr.ordinal(target=targetElo, alpha=200 / model.sigma)
            racer.races.append({'raceID': raceRow['raceID'], 'raceNum': raceRow['raceNum'], 'round':  raceRow['round'], 
                                'pos': type,
                                'type':'womens' if womens else 'open',
                                'opponentTeam': raceRow['teamAName'], 
                                'opponentNick': raceRow['teamANick'],
                                'score': raceRow['teamBScore'],
                                'outcome': raceRow['teamBOutcome'], 
                                'newRating': newRating})
        
        # print(ratings)
    # if i >= 20:
    #     break

not enough sailors in this race, skipping s16/2016-jeremy-mcintyre-team-race/41
not enough sailors in this race, skipping s16/2016-jeremy-mcintyre-team-race/41
not enough sailors in this race, skipping s16/mendelblatt-team-race/55
not enough sailors in this race, skipping s16/mendelblatt-team-race/55
not enough sailors in this race, skipping s16/mendelblatt-team-race/59
not enough sailors in this race, skipping s16/mendelblatt-team-race/59
not enough sailors in this race, skipping s16/mendelblatt-team-race/62
not enough sailors in this race, skipping s16/mendelblatt-team-race/62
not enough sailors in this race, skipping s16/mendelblatt-team-race/67
not enough sailors in this race, skipping s16/mendelblatt-team-race/67
not enough sailors in this race, skipping s16/nw-team-race/6
not enough sailors in this race, skipping s16/nw-team-race/6
not enough sailors in this race, skipping s16/nw-team-race/7
not enough sailors in this race, skipping s16/nw-team-race/7
not enough sailors in this r

In [163]:
eligible_skippers = [p for p in people.values()
                    if targetSeason in p.seasons['Skipper']]
eligible_crews = [p for p in people.values()
                    if targetSeason in p.seasons['Crew']]

for p in people.values():
    p.skipperRankTR = 0
    p.crewRankTR = 0
    p.womenSkipperRankTR = 0
    p.womenCrewRankTR = 0

for i,s in enumerate(sorted([p for p in eligible_skippers if p.tsr.mu != model.mu], key=lambda p: p.tsr.ordinal(), reverse=True)):
    s.skipperRankTR = i + 1
for i,s in enumerate(sorted([p for p in eligible_crews if p.tcr.mu != model.mu], key=lambda p: p.tcr.ordinal(), reverse=True)):
    s.crewRankTR = i + 1

for i,s in enumerate(sorted([p for p in eligible_skippers if p.wtsr.mu != model.mu], key=lambda p: p.wtsr.ordinal(), reverse=True)):
    s.womenSkipperRankTR = i + 1
for i,s in enumerate(sorted([p for p in eligible_crews if p.wtcr.mu != model.mu], key=lambda p: p.wtcr.ordinal(), reverse=True)):
    s.womenCrewRankTR = i + 1

allRows = []
for sailor,p in people.items():
    allRows.append([p.name, p.teams, p.skipperRankTR, p.womenSkipperRankTR, sailor, p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma),p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma), p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma),  p.year, p.links, p.tsr.mu, p.tcr.mu,len(p.races), p.races])

df_fullsailors = pd.DataFrame(allRows, columns=['Sailor', 'Teams', 'TRSkipperRank', 'TRWomenSkipperRank', 'key', 'SkipperOrdinal', 'WomenSkipperOrdinal', 'CrewOrdinal',  'GradYear', 'Links', 'SkipperMU','CrewMU','numRaces', 'Races'])
df_fullsailors 

,Sailor,Teams,TRSkipperRank,TRWomenSkipperRank,key,SkipperOrdinal,WomenSkipperOrdinal,CrewOrdinal,GradYear,Links,SkipperMU,CrewMU,numRaces,Races
0,Hugh MacGillivray,[Connecticut College],0,0,hugh-macgillivray,1607.294382,1000.0,1134.523025,18,[hugh-macgillivray],35.115145,26.961009,161,"[{'raceID': 's16/john-jackson-team-racce/4', '..."
1,Eliza Garry,[Connecticut College],0,0,eliza-garry,1060.426487,1000.0,1396.587001,17,[eliza-garry],26.035230,31.540925,73,"[{'raceID': 's16/john-jackson-team-racce/4', '..."
2,Carter Pemberton,[Connecticut College],0,0,carter-pemberton,1619.668418,1000.0,1119.312824,20,[carter-pemberton],33.102154,26.647779,266,[{'raceID': 'f16/hap-moore-team-race-backup/1'...
3,Paige Dunlevy,[Connecticut College],0,0,paige-dunlevy,1000.000000,1000.0,1315.814466,20,[paige-dunlevy],25.000000,26.933513,114,[{'raceID': 'f16/hap-moore-team-race-backup/1'...
4,Ragna Agerup,[Connecticut College],0,0,ragna-agerup,1082.236601,1000.0,1000.000000,20,[ragna-agerup],22.315331,25.000000,21,[{'raceID': 'f16/hap-moore-team-race-backup/1'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151,Anthony Torrez,[Southern Cal],0,0,anthony-torrez,1000.000000,1000.0,1021.141099,25,[anthony-torrez],25.000000,24.509752,4,"[{'raceID': 's24/mustang-team-race/2', 'raceNu..."
4152,Jack Blackman,[Southern Cal],357,0,jack-blackman,996.280052,1000.0,1000.000000,24,[jack-blackman],23.468284,25.000000,4,"[{'raceID': 's24/mustang-team-race/2', 'raceNu..."
4153,Genaro Flores - Machorro,[Southern Cal],0,0,genaro-flores-machorro,1000.000000,1000.0,1021.141099,25,[genaro-flores-machorro],25.000000,24.509752,4,"[{'raceID': 's24/mustang-team-race/2', 'raceNu..."
4154,Maggie McEachen,[UC Santa Cruz],0,0,maggie-mceachen,1000.000000,1000.0,887.746351,27,[maggie-mceachen],25.000000,18.954784,4,"[{'raceID': 's24/mustang-team-race/3', 'raceNu..."


In [157]:
allTeams = []
for teamName in teamRegions.keys():
    filtered_people = [p for p in people.values() if teamName in p.teams]
    
    members = [{'name': p.name, 'key': p.key, 'teams': p.teams, 
                'tsr': p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                'wtsr': p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                'tcr': p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                'wtcr': p.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                'seasons': p.seasons
                } for p in filtered_people]
    
    numTops = 3
    topSkippers = sorted([p['tsr'] for p in members 
                          if p['tsr'] != 1000
                          and targetSeason in p['seasons']['Skipper']
                          ], reverse=True)[:numTops]
    topCrews = sorted([p['tcr'] for p in members if p['tcr'] != 1000 and targetSeason in p['seasons']['Crew']], reverse=True)[:numTops]
    
    avgTopSkippers = sum(topSkippers) / numTops
    topRating = (sum(topSkippers) + sum(topCrews)) / (numTops * 2)
    
    numTops = 3
    topWomenSkippers = sorted([p['wtsr'] for p in members if p['wtsr'] != 1000 and targetSeason in p['seasons']['Skipper']], reverse=True)[:numTops]
    topWomenCrews = sorted([p['wtcr'] for p in members if p['wtcr'] != 1000 and targetSeason in p['seasons']['Crew']], reverse=True)[:numTops]
    
    topWomenRating = (sum(topWomenSkippers) + sum(topWomenCrews)) / (numTops * 2)
    
    allTeams.append({'teamName': teamName, 'topRating': topRating,'topWomenRating': topWomenRating, 'avgTopSkippers': avgTopSkippers, 'topSkippers': topSkippers, 'topCrews': topCrews})

df_teams = pd.DataFrame(allTeams)
df_teams  

,teamName,topRating,topWomenRating,avgTopSkippers,topSkippers,topCrews
0,Hawaii,1164.680863,1193.337895,1163.457990,"[1301.190191836494, 1115.3321116236211, 1073.8...","[1331.9514130332575, 1082.9756857785394, 1082...."
1,Brown,1632.961318,1547.241992,1687.286475,"[1733.7119947423093, 1723.0846899203775, 1605....","[1706.2930456050244, 1528.6906642319393, 1500...."
2,Southern Cal,1008.710575,0.000000,996.280052,"[996.2800517264889, 996.2800517264889, 996.280...","[1021.1410986533737, 1021.1410986533737, 1021...."
3,Salve Regina,1019.498118,504.729168,1020.611398,"[1095.6711471821648, 1004.9956480553317, 961.1...","[1048.3779352961903, 1010.9560084646141, 995.8..."
4,UC Santa Barbara,1335.203547,197.482491,1363.764787,"[1516.681910438604, 1287.3062257216397, 1287.3...","[1363.598588902803, 1332.5014970038642, 1223.8..."
...,...,...,...,...,...,...
202,Cleveland State,0.000000,0.000000,0.000000,[],[]
203,Sewanee,0.000000,0.000000,0.000000,[],[]
204,Case Western,0.000000,0.000000,0.000000,[],[]
205,Oklahoma,0.000000,0.000000,0.000000,[],[]


In [179]:
print([r for r in people['ellie-pinto'].races if r['pos'] == 'Skipper'])
print([r for r in people['matthew-elliott'].races if r['raceID'] == 's24/41st-cardinal-red-lynne-marchiando/62'])
# print([r for r in people['wendy-wu'].races if r['raceID'] == 's24/41st-cardinal-red-lynne-marchiando/59'])
print([r for r in people['jemma-schroder'].races if r['raceID'] == 's24/41st-cardinal-red-lynne-marchiando/62'])
print([r for r in people['yitian-zhu'].races if r['raceID'] == 's24/41st-cardinal-red-lynne-marchiando/62'])

[]
[{'raceID': 's24/41st-cardinal-red-lynne-marchiando/62', 'raceNum': 62, 'round': 'Round 3', 'pos': 'Skipper', 'type': 'open', 'opponentTeam': 'Harvard', 'opponentNick': 'Crimson', 'score': '3-4-6', 'outcome': 'lose', 'newRating': 1120.0011553112263}, {'raceID': 's24/41st-cardinal-red-lynne-marchiando/62', 'raceNum': 62, 'round': 'Round 3', 'pos': 'Skipper', 'type': 'open', 'opponentTeam': 'Harvard', 'opponentNick': 'Crimson', 'score': '3-4-6', 'outcome': 'lose', 'newRating': 1120.0011553112263}]
[{'raceID': 's24/41st-cardinal-red-lynne-marchiando/62', 'raceNum': 62, 'round': 'Round 3', 'pos': 'Crew', 'type': 'womens', 'opponentTeam': 'Harvard', 'opponentNick': 'Crimson', 'score': '3-4-6', 'outcome': 'lose', 'newRating': 1119.4939361820177}]
[{'raceID': 's24/41st-cardinal-red-lynne-marchiando/62', 'raceNum': 62, 'round': 'Round 3', 'pos': 'Crew', 'type': 'womens', 'opponentTeam': 'Harvard', 'opponentNick': 'Crimson', 'score': '3-4-6', 'outcome': 'lose', 'newRating': 1154.520355401600